Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
    http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License. 
 
# MONAI Label Bundle- Pancreas and Tumor Segmentation

***Bring Your Own Data to MONAI Label with Pancreas and Tumor Segmentation***

In this notebook, we provide a step-by-step setup of using the local dataset for MONAI Label. The monaibundle app with pancreas and tumor segmentation is demonstrated. With the released `pancreas_ct_dints_segmentation` bundle and 3D Slicer as the client viewer, we provide detailed guidance on auto segmentation, annotation edit, training and active learning. 


**pancreas_ct_dints_segmentation**

This bundle uses a neural architecture search algorithm (DiNTS)[1] for volumetric (3D) segmentation of the pancreas and pancreatic tumor from CT images.

The inference and training require at least 16GB-memory GPUs.


**Modality:** CT

The method framework for pancreas and tumor segmentation with MONAI Label shown in 3D Slicer

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/dints.png" alt="drawing" width="700"/></p>
<p align = "center">This model is trained using the neural network model from the neural architecture search algorithm, DiNTS.</p>



## 1. Prepare MONAI Label and Dataset

### Prerequisites
- **Install MONAI Label** weekly preview release: 

In [ ]:
!pip install monailabel-weekly

In this tutorial, we show simple steps to create a local dataset with MSD Task07 Pancreas and tumor data in the local machine. 

### Prepare Pancreas CT Scans

Download MSD Pancreas dataset as the sample dataset using monailabel API. The task is the volumetric (3D) segmentation of the pancreas and tumor from CT image. The segmentation of pancreas/tumor is formulated as the voxel-wise 3-class classification. Each voxel is predicted as either foreground (1:Pancreas, 2:Tumor) or background. The dataset is from the 2018 MICCAI challenge.

In [ ]:
!monailabel datasets --download --name Task07_Pancreas --output datasets

## 2. Start MONAI Label - monaibundle app 

In this tutorial, we use the monaibundle app for demonstrating the bundle models with a segmentation task.

### 2.1 Get the monaibundle App
MONAI Label provides the API to fetch the sample app, type the `moniabundle` as the name and specify the saved directory path `apps`.


In [ ]:
!monailabel apps --download --name monaibundle --output apps

### 2.2 Starting MONAI Label Server

Specify the bundle name in --conf models <BUNDLENAME> argument. For example pancreas_ct_dints_segmentation.
    
    
In the `monailabel start_server` command, `--app` argument sets the sample-app (monaibundle) path, `--studies` specifies the mounted data
directory path, and `--conf models` loads the bundle config directory. The above command uses `pancreas_ct_dints_segmentation` bundle as an example. The bundle directory should be placed in `model` folder of the monaibundle app.
    
Set your gihub auth token to get authentication and avoid rate limit of MODEL ZOO access. 
    
```bash
   --conf auth_token <Github Auth Token> 
```

In [ ]:
!monailabel start_server --app apps/monaibundle --studies datasets/Task07_Pancreas/imagesTr --conf models pancreas_ct_dints_segmentation --conf auth_token <Github Auth Token>

## 3. Pancreas Auto Segmentation with 3D Slicer

Below is a step-by-step tutorial on using 3D Slicer for the auto pancreas and tumor segmentation with the DiNTS bundle. Pre-trained model is provided by the bundle and loaded by MONAI Label server command.


**Use 3D Slicer and MONAI Label Plugin**: the tutorial starts with monaibundle app in the 3D Slicer, please refer to the ["Hello, World!" tutorial](monailabel_HelloWorld_radiology_3dslicer.ipynb) for detailed instructions on installing 3D Slicer and MONAI Label plugin.

### 3.1 Select Model and Fetch Data

- In the Slicer, click the MONAI Label plugin with the icon. First, users can select the bundle model loaded with MONAI Label server, such as "**pancreas_ct_dints_segmentation**" in this tutorial. 

Click **next sample** to load image data from Datastore.

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas1.png" alt="drawing" width="700"/></p>


### 3.2 Run Pancreas/Tumor Inference

- The bundle contains a DiNTS pre-trained model weights, on the pancreas and tumor segmentation task, click **run** to run inference.  

- The pre-trained model by DiNTS algorithm is in the bundle's "**models**" folder: "**apps/monaibundle/model/{BUNDLENAME}/models/model.pt**".

Users can monitor the logs in the above MONAI Label start server cell.

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas2.png" alt="drawing" width="700"/></p>

### 3.3 Edit Annotations 

- Users can edit the inference labels manually using Segment Editor after auto inference. 

#### Corrections with Segment Editor:

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas3.png" alt="drawing" width="700"/></p>

### 3.4 Submit Label

- Click **Submit Label** to save the ground truth label to the file system. The final annotations will be saved to "**labels/final**" folder in the local dataset, in this tutorial, the annotated label will be saved to "**datasets/Task07_Pancreas/imagesTr/labels/final**"

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas4.png" alt="drawing" width="700"/></p>


## 4. Fine-Tune Model with 3D Slicer

MONAI Label offers **active learning** strategies and can interactively fine-tune models. Users can train their models once new annotated labels are saved, with any number of finished labels. MONAI Label server set train and validation split percentage to 0.2 by default. It's recommended to label five or more before running training. Click **Train** button, MONAI Label server will fetch saved final ground truth label and fine-tune the prior model. 

Users can monitor the logs in the MONAI Label server terminal

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas5.png" alt="drawing" width="700"/></p>

The active learning process will select unlabeled batch data, users can choose strategies in the Slicer UI.

In the second round of labeling, when next image is selected, those trained images will be marked as labeled data, and won't be selected. Active learning strategies such as "first/random" will be used for selecting which unlabeled data. 



## 4. Repeat Until All Data are Annotated and Trained 

Iteratively label and train samples until all unlabeled data are annotated. The model will be fine-tuned with all labeled data at the end. 

For each training loop, the new best metric model will be saved in "**model/model.pt**", in this use case, "**apps/monaibundle/{BUNDLENAME}/model/model.pt**" is updated and saved.

## 5. Conclusion

This tutorial demonstrates pancreas and tumor segmentation in MONAI Label. The pancreas and tumor segmentation with the DiNTS bundle is shown. The step-by-step tutorial on starting MONAI Label server with the downloaded bundle, auto inference, segmentation editor, submit labels, and training process is introduced. MONAI Label and its apps highlight the robustness of adapting users' local data, and the workflow can be used to extend data and model deployments. 

Reference:

[1] He, Y., Yang, D., Roth, H., Zhao, C. and Xu, D., 2021. Dints: Differentiable neural network topology search for 3d medical image segmentation. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 5841-5850).